## Построение оптимального маршрута полётной миссии БПЛА для обследования установленной зоны с помощью генетического алгоритма

Импортируем необходимые модули.

In [2]:
import time
import logging
import numpy as np

from survey_route_generation.data.vehicle_data import VehicleData
from survey_route_generation.data.mission_settings import MissionSettings
from survey_route_generation.factories.route_generator_factory import RouteGeneratorFactory
from survey_route_generation.scaffolding.geojson import save_result
from survey_route_generation.scaffolding.notebook import show_route_result
from survey_route_generation.data.data_keeper import DataKeeper
from survey_route_generation.scaffolding.logging import tune_logging
from survey_route_generation.scaffolding.dirs import LOGS_DIR
from survey_route_generation.scaffolding.dirs import DATA_DIR

from config import settings

Настроим логирование.

In [3]:
# Настраиваем логирование
tune_logging(settings.console_log, settings.file_log, LOGS_DIR)

Зададим параметры БПЛА и зоны обследования.

In [4]:
# Параметры БПЛА: ширина приборного зрения (м)
vehicle_data = VehicleData(settings.vision_width)

# Настройки полётной миссии: координаты точки начала и точки завершения миссии
mission_settings = MissionSettings(
    settings.start_point,
    settings.end_point
)

# Координаты точек зоны обследования
survey_area_points = np.array(settings.survey_area_points)

Зададим параметры определения оптимального маршрута для генетического алгоритма.

In [5]:
generator_factory = RouteGeneratorFactory()

# Настраиваем экспорт данных о промежуточных вычислениях
data_keeper = DataKeeper(DATA_DIR)

data_keep_func = None
if settings.export_data:
    data_keep_func = data_keeper.keep

# Размер начальной популяции
generator_factory.population_size = settings.population_size
# Доля выживших особей при отборе
generator_factory.selection_rate = settings.selection_rate
# Количество генотипов при размножении
generator_factory.parents_count = settings.parents_count
# Доля мутированных особей
generator_factory.mutants_rate = settings.mutants_rate
# Максимальное число циклов смены популяции
generator_factory.max_lifecycles = settings.max_lifecycles
# Тип выбора родительских генотипов при скрещивании: panmixia, inbreeding, outbreeding
generator_factory.parents_choice_type = settings.parents_choice_type
# Тип сравнения генотипов при группировке родителей методами аутбридинга и инбридинга: fitness, combination
generator_factory.parents_similarity_type = settings.parents_similarity_type
# Число мутаций в генотипе: доля или конкретное число
generator_factory.mutation_swap_value = settings.mutation_swap_value
# Тип числа мутаций: rate, value
generator_factory.mutation_swap_type = settings.mutation_swap_type
# Значимость сокращения длины маршрута при оценке приспособленности
generator_factory.route_distance_weight = settings.route_distance_weight
# Значимость увеличения плавности движения при оценке приспособленности
generator_factory.route_turns_angle_weight = settings.route_turns_angle_weight
# Значимость количества самопересечений маршрута при оценке приспособленности
generator_factory.route_self_intersection_weight = settings.route_self_intersection_weight
# Применять ли к генотипам правило "ближайших точек"
generator_factory.repair_route_genotypes = settings.repair_route_genotypes

# Задаём функцию сохранения данных для экспорта
generator_factory.data_keep_func = data_keep_func

Создадим генератор полётной миссии с данными параметрами и запустим поиск маршрута.

In [6]:
# Создаём экземпляр генератора маршрута
generator = generator_factory.make()

# Выполняем генерацию маршрута
start_time = time.time()
route_result = generator.generate_route(
    vehicle_data,
    mission_settings,
    survey_area_points
)
logging.info("Затрачено времени: \t" + str(time.time() - start_time))
logging.info("Hash маршрута: \t" + str(route_result["route_hash"]))
logging.info("Приспособленность маршрута: \t" + str(route_result["route_fitness"]))

INFO:root:Точка входа в зону обследования: 	[54.51709599 18.55997538]
INFO:root:Точка выхода из зоны обследования: 	[54.59195848 18.80996704]
INFO:root:Границы описанного прямоугольника зоны обследования: 	{'lat_bot': 54.51709598731107, 'lat_top': 54.78762142424453, 'lon_left': 18.39307164274357, 'lon_right': 18.80996704101563}
INFO:root:Расстояние между парой ключевых точек: 	2850.0
INFO:root:Количество ключевых точек в начальной сетке: 	132
INFO:root:Количество ключевых точек в маршруте: 	45
INFO:root:Эволюционный цикл: 	0
INFO:root:Размер популяции: 	32
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	2.434049746767435	2.739074666212195	2.9404300331789868
INFO:root:Эволюционный цикл: 	1
INFO:root:Размер популяции: 	31
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	2.6976317612519507	3.114637986944979	3.6579289334143494
INFO:root:Эволюционный цикл: 	2
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное зн

INFO:root:Эволюционный цикл: 	40
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	3.5499602106394352	3.704288247198943	3.729294986862344
INFO:root:Эволюционный цикл: 	41
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	3.648950542038181	3.7115618336670693	3.729294986862344
INFO:root:Эволюционный цикл: 	42
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	3.602114450316422	3.705560717757126	3.729294986862344
INFO:root:Эволюционный цикл: 	43
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	3.645640984273173	3.7099979169617234	3.729294986862344
INFO:root:Эволюционный цикл: 	44
INFO:root:Размер популяции: 	30
INFO:root:Минимальное, среднее и максимальное значения приспособленности: 	3.6248087168488063	3.709575213391205	3.729294986862344
INFO:root:Эволюционный цик

При необходимости выгрузим промежуточные данные и сохраним полученный маршрут в файл формата GeoJson.

In [7]:
# Выгружаем данные промежуточных вычислений
if settings.export_data:
    data_keeper.keep(route_result, "result_obtaining")
    data_keeper.save()

# Сохраняем найденный маршрут в формате GeoJson
save_result(route_result, survey_area_points, mission_settings)

4004

И отобразим полученный результат на карте.

In [ ]:
show_route_result(
    mission_settings.start_point,
    survey_area_points.tolist(),
    mission_settings.end_point,
    route_result
)